<a href="https://colab.research.google.com/github/hyesungKomet/deep_learning/blob/main/ch3_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 마운트

In [ ]:
%cd /content/drive/MyDrive/machine_learning

/content/drive/MyDrive/machine_learning


In [ ]:
%cd deep-learning-from-scratch-2

/content/drive/My Drive/machine_learning/deep-learning-from-scratch-2


In [ ]:
import sys
sys.path.append('..')
from common.util import preprocess
import numpy as np

## 전처리

In [ ]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text) #전처리
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


## 맥락, 타겟 생성 함수

In [ ]:
def create_contexts_target(corpus, window_size = 1):
  target = corpus[window_size:-window_size] # 1부터 -1 : 두번째~마지막두번째
  contexts = []

  for idx in range(window_size, len(corpus) - window_size):
    cs = []
    for t in range(-window_size, window_size + 1):
      if t==0: #target에 해당하는 id는 제외
        continue
      cs.append(corpus[idx+t]) #[0,2], [1.3] 등등이 됨
    contexts.append(cs) #위의 1차원 배열이 추가되어 2차원이 됨

  return np.array(contexts), np.array(target) # contexts는 s를 꼭 붙이기!

In [ ]:
contexts, target = create_contexts_target(corpus, window_size=1)

print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


## 원핫 표현으로 변경

In [ ]:
from common.util import convert_one_hot # 단어 id 목록, 어휘 수를 인수로!

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [ ]:
print(target) # 6x7
print()
print(contexts) # 6x2x7

[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]

[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]


## simpleCBOW

In [ ]:
from common.layers import MatMul, SoftmaxWithLoss

In [ ]:
class simpleCBOW:
  def __init__(self, vocab_size, hidden_size):
    V, H = vocab_size, hidden_size

    # 가중치 초기화
    W_in = 0.01 * np.random.randn(V,H).astype('f') # float로 형 바꾸기 - 단어의 분산표현
    W_out = 0.01 * np.random.randn(H,V).astype('f') # 32비트 부동소수점 수로 초기화

    # 계층 생성
    self.in_layer0 = MatMul(W_in)
    self.in_layer1 = MatMul(W_in)
    self.out_layer = MatMul(W_out)
    self.loss_layer = SoftmaxWithLoss()

    # 가중치, 기울기 리스트에 담기
    layers = [self.in_layer0, self.in_layer1, self.out_layer]
    self.params, self.grads = [], []
    for layer in layers:
      self.params += layer.params
      self.grads += layer.grads

    # 인스턴스 변수에 단어의 분산표현 저장
    self.word_vecs = W_in

  def forward(self, contexts, target): #contexts는 3차원, target은 2차원 배열로 가정
    h0 = self.in_layer0.forward(contexts[:, 0]) # 정답레이블 기준 왼쪽 단어
    h1 = self.in_layer1.forward(contexts[:,1]) #정답레이블 기준 오른쪽 단어
    h = (h0 + h1) * 0.5
    score = self.out_layer.forward(h)
    loss = self.loss_layer.forward(score, target)
    return loss

  def backward(self, dout = 1):
    ds = self.loss_layer.backward(dout) # 각각 MulMax, SoftmaxwithLoss의 backward 메소드 사용 - 기울기 갱신됨
    da = self.out_layer.backward(ds)
    da *= 0.5
    self.in_layer1.backward(da)
    self.in_layer0.backward(da)
    return None 

## 옵티마이저 처리시 같은 가중치가 계속 쓰이기에 중복 제거해줌
매개변수 배열 중 중복되는 가중치를 하나로 모아
    그 가중치에 대응하는 기울기를 더한다.

## 학습

In [ ]:
from common.trainer import Trainer
from common.optimizer import Adam


In [ ]:
window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [ ]:
model = simpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

## 학습 점검

In [ ]:
word_vecs = model.word_vecs #word_vecs에 W_in, 즉 단어의 분산표현 저장됨

for word_id, word in id_to_word.items(): #딕셔너리에서 key인 id와 value인 단어
  print(word, word_vecs[word_id])

you [-0.7735876  -0.8379491   0.77557456  0.8222838  -0.80081767]
say [-0.00403866  1.1611474   0.39305916 -1.1360828   1.0862681 ]
goodbye [-0.98396957 -0.05104873  0.9712033   0.1980111  -0.88927454]
i [-0.93692654 -0.1067699   1.0296179   0.6362648   0.10957809]
hello [-0.78331304 -0.8412493   0.75901616  0.8326869  -0.8260973 ]
. [ 1.0314195   0.85285246 -1.0348856  -0.86508304  0.87489253]
